# アユムのファイル

In [4]:
!pip install ultralytics

In [5]:
import numpy as np
from ultralytics import YOLO
import cv2
import os

# Load a model (既に読み込んでいる場合は重複しないよう調整してください)
model = YOLO("yolo11n-pose.pt")

# --- 推論用設定 ---
# 入力例: 画像ファイル, 動画ファイル, フォルダ ("images/"), またはカメラ(0)
source = "/2011tokyo_mister_fp-011-320x480.jpg"    # ここを使いたい入力に変更
imgsz = 640
conf = 0.25
save_dir = "runs/pose_infer"   # 保存先ディレクトリ

os.makedirs(save_dir, exist_ok=True)

# 推論実行
results = model.predict(source=source, imgsz=imgsz, conf=conf)

# results は iterable。複数フレーム/画像に対して処理する。
for i, r in enumerate(results):
    # 可視化画像を取得して保存 (r.plot() はRGB画像を返す)
    try:
        annotated = r.plot()  # numpy array (RGB)
        out_path = os.path.join(save_dir, f"annotated_{i}.jpg")
        # OpenCV はBGRで扱うため変換して保存
        cv2.imwrite(out_path, cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR))
        print(f"Saved annotated image: {out_path}")
    except Exception as e:
        print("Could not plot/save annotated image:", e)

    # キーポイントがあれば取り出して表示 (モデルの出力形式に依存するため安全にアクセス)
    try:
        # ultralytics のバージョンにより属性名が異なることがあるので安全に取得
        if hasattr(r, "keypoints") and r.keypoints is not None:
            kps = r.keypoints
        elif hasattr(r, "masks") and r.masks is not None:
            kps = r.masks
        else:
            kps = None

        if kps is not None:
            # kps を表示しやすい形に変換して出力
            try:
                arr = np.array(kps)  # 多くの場合 numpy に変換可能
            except Exception:
                arr = kps
            print(f"Frame {i} keypoints / mask info:", type(arr), getattr(arr, "shape", None))
            # 必要ならここで arr を解析して各姿勢のキーポイント座標を処理
        else:
            print(f"Frame {i}: no keypoints found in result")
    except Exception as e:
        print("Error while extracting keypoints:", e)
# ...existing code...

Speed: 8.3ms preprocess, 52.4ms inference, 7.3ms postprocess per image at shape (1, 3, 640, 448)
Saved annotated image: runs/pose_infer/annotated_0.jpg
Frame 0 keypoints / mask info: <class 'ultralytics.engine.results.Keypoints'> torch.Size([1, 17, 3])
